This is a page for exploring a single STAC collection

In [ ]:
import geoviews as gv
import panel as pn
import param

import utils.cards
import utils.location
import utils.template
import utils.xyt
import utils.zarr

pn.extension()
gv.extension("bokeh")

In [ ]:
class Page(utils.location.UrlQueryParams):

    point_of_interest: utils.xyt.XYT | None = param.ClassSelector(class_=utils.xyt.XYT, allow_None=True, default=None)  # type: ignore
    dataset: utils.zarr.ZarrDataset | None = param.ClassSelector(class_=utils.zarr.ZarrDataset, allow_None=True, default=None)  # type: ignore

    @param.depends("site", "collection", watch=True)
    def update_poi_dataset(self):
        if self.site is None or self.collection is None:
            self.point_of_interest = None
            self.dataset = None
            return
        extent = utils.xyt.Extent.from_pystac(self.collection.extent)
        self.point_of_interest = utils.xyt.XYT(extent=extent)
        self.dataset = utils.zarr.ZarrDataset.from_pystac(self.point_of_interest, self.collection)

    @param.depends("site", "collection", "point_of_interest", "dataset", watch=False)
    def main(self) -> pn.Column:
        """content to render in the main area of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        elif self.collection is None:
            elements.append(pn.pane.Alert("Collection not found", alert_type="danger"))
        elif self.point_of_interest is None:
            # should be unreachable
            elements.append(pn.pane.Alert("Point of interest not found", alert_type="danger"))
        elif self.dataset is None:
            # should be unreachable
            elements.append(pn.pane.Alert("Dataset not found", alert_type="danger"))
        else:
            site_name = self.site.title or self.site.id
            collection_name = self.collection.title or self.collection.id
            elements.append(
                pn.pane.Markdown(
                    f"""
                    # Data explorer

                    On this page you can interactively explore our data collection {collection_name}
                    for the site {site_name}.
                    """
                )
            )
            elements.append(pn.pane.Markdown("## Site"))
            elements.append(utils.cards.site(self.site, collapsed=True))
            elements.append(pn.pane.Markdown("## Collection"))
            elements.append(utils.cards.collection(self.site, self.collection, collapsed=True))
            elements.append(pn.pane.Markdown(
                f"""
                ## Data

                The controls below allow you to explore this datacube in time and space.
                You can enter a point of interest (latitude and longitude) in the form provided,
                or click on the map to select a point.
                You can also select a date of interest using the date slider,
                or by clicking on the time series plot.
                
                The map next to the point of interest controls is shown in the web mercator projection.
                However, this collection is in the EPSG:{self.dataset.crs.to_epsg()} coordinate reference system,
                and is plotted in that projection below.
                """
            ))
            elements.append(self.dataset)
            elements.append(self.dataset.download_buttons())

        return pn.Column(
            *elements,
        )
    
    @param.depends("site", "collection", "point_of_interest", "dataset", watch=False)
    def sidebar(self) -> pn.Column:
        """content to render in the sidebar of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        elif self.collection is None:
            elements.append(pn.pane.Alert("Collection not found", alert_type="danger"))
        elif self.point_of_interest is None:
            # should be unreachable
            elements.append(pn.pane.Alert("Point of interest not found", alert_type="danger"))
        elif self.dataset is None:
            # should be unreachable
            elements.append(pn.pane.Alert("Dataset not found", alert_type="danger"))
        else:
            elements.append(self.point_of_interest)
            elements.append(self.dataset.widgets())

        return pn.Column(*elements)

In [ ]:
page = Page()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(page, {"site_id": "site-id", "collection_id": "collection-id"})  # type: ignore

In [ ]:
template = utils.template.get_template(main=page.main, sidebar=page.sidebar)
template.servable();